In [1]:
import wrds
import pandas as pd
conn=wrds.Connection()

Enter your WRDS username [isabella]:ic17
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [2]:
sp500 = conn.raw_sql("""
                        select a.permno
                        from crsp.msp500list as a,
                        crsp.msf as b 
                        where a.permno = b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date >= '01/01/2018' and b.date <= '12/31/2022'
                        group by a.permno
                        HAVING COUNT(DISTINCT CONCAT(EXTRACT(YEAR FROM b.date), '-', EXTRACT(QUARTER FROM b.date))) = 20
                        ORDER BY a.permno;
                        """)

In [3]:
mse = conn.raw_sql("""
                        select comnam, ncusip, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """)

# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')


In [4]:
ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno,
                  linktype, linkprim
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """)

# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])


In [5]:
sp500ccm

,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,linktype,linkprim
0,10104.0,ORACLE SYSTEMS CORP,68389X10,11.0,3.0,7372.0,ORCL,012142,01,LC,P
1,10104.0,ORACLE CORP,68389X10,11.0,3.0,7372.0,ORCL,012142,01,LC,P
2,10104.0,ORACLE CORP,68389X10,11.0,3.0,7372.0,ORCL,012142,01,LC,P
3,10104.0,ORACLE CORP,68389X10,11.0,1.0,7372.0,ORCL,012142,01,LC,P
4,10107.0,MICROSOFT CORP,59491810,11.0,3.0,7370.0,MSFT,012141,01,LC,P
...,...,...,...,...,...,...,...,...,...,...,...
4093,93096.0,DOLLAR GENERAL CORP NEW,25667710,11.0,1.0,5331.0,DG,004016,02,LC,P
4094,93429.0,C B O E HOLDINGS INC,12503M10,11.0,3.0,6211.0,CBOE,184500,01,LC,P
4095,93429.0,C B O E GLOBAL MARKETS INC,12503M10,11.0,3.0,6211.0,CBOE,184500,01,LC,P
4096,93429.0,C B O E GLOBAL MARKETS INC,12503M10,11.0,5.0,6211.0,CBOE,184500,01,LC,P


In [6]:
sp500ccm = sp500ccm.drop(columns=['linktype', 'linkprim'])
sp500ccm = sp500ccm[['permno', 'comnam', 'ncusip',\
                     'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid']]

In [7]:
names = conn.raw_sql(""" select gvkey, cik, sic, naics, gind, gsubind from comp.names """)

# Merge sp500 constituents table with names table
sp500cik = pd.merge(sp500ccm, names, on='gvkey',  how='left')
sp500cik.head()

,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,cik,sic,naics,gind,gsubind
0,10104.0,ORACLE SYSTEMS CORP,68389X10,11.0,3.0,7372.0,ORCL,012142,01,0001341439,7370,519290,451030,45103020
1,10104.0,ORACLE CORP,68389X10,11.0,3.0,7372.0,ORCL,012142,01,0001341439,7370,519290,451030,45103020
2,10104.0,ORACLE CORP,68389X10,11.0,3.0,7372.0,ORCL,012142,01,0001341439,7370,519290,451030,45103020
3,10104.0,ORACLE CORP,68389X10,11.0,1.0,7372.0,ORCL,012142,01,0001341439,7370,519290,451030,45103020
4,10107.0,MICROSOFT CORP,59491810,11.0,3.0,7370.0,MSFT,012141,01,0000789019,7372,513210,451030,45103020


In [8]:
sp500_ciks = sp500cik.cik.unique()

In [9]:
sp500_ciks

array(['0001341439', '0000789019', '0001113169', '0000773840',
       '0000007084', '0000798354', '0000021344', '0000813672',
       '0001047862', '0000818479', '0000815556', '0000936340',
       '0000028385', '0001551182', '0000719739', '0000034088',
       '0000789570', '0000823768', '0000040533', '0000040545',
       '0000920148', '0001091667', '0001489393', '0001467858',
       '0001466258', '0000051143', '0001506307', '0000860730',
       '0001501585', '0001510295', '0001519751', '0001524472',
       '0001521332', '0001534701', '0001326801', '0000077360',
       '0001551152', '0001513761', '0001555280', '0000077476',
       '0000764180', '0001478242', '0001163165', '0001564708', nan,
       '0000318154', '0000087347', '0001579241', '0001585689',
       '0000093410', '0000320193', '0000006951', '0001601712',
       '0000759944', '0000101778', None, '0001396009', '0001637459',
       '0001633917', '0000097476', '0001645590', '0000827052',
       '0001659166', '0000047111', '00000568

In [10]:
len(sp500_ciks)

429

In [ ]:
from sec_edgar_downloader import Downloader
dl = Downloader("NYU", "yc3815@nyu.edu",'/Users/isabella/Desktop/tickers')
for equity_id in sp500_ciks:
    dl.get('10-Q', equity_id, after="2018-01-01", before="2022-12-31")
    dl.get('10-K', equity_id, after="2018-01-01", before="2022-12-31")

In [14]:
import os

# Specify the path to the folder on your desktop
folder_path = "/Users/isabella/Desktop/tickers/sec-edgar-filings"

# Get a list of all folder names inside the specified directory
folder_names = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]

print(folder_names)

['0000882095', '0000943819', '0001166691', '0000723254', '0000909832', '0000046080', '0000016732', '0001001082', '0000100517', '0001478242', '0001039684', '0000726728', '0000091419', '0000106640', '0000813828', '0000718877', '0001652044', '0000014693', '0001037540', '0000354950', '0000217346', '0001047862', '0001521332', '0000006951', '0001524472', '0001038357', '0000029989', '0001688568', '0000874716', '0001020569', '0001109357', '0000087347', '0000077360', '0000018230', '0001467858', '0000027419', '0000009389', '0001701605', '0000788784', '0000712515', '0000813672', '0000920522', '0000765880', '0000035527', '0001385157', '0000732717', '0001032208', '0001071739', '0001442145', '0000354190', '0001601712', '0000002969', '0000920148', '0001126328', '0000916076', '0000040545', '0001024478', '0000882184', '0000914208', '0000064803', '0001466258', '0000352915', '0001137774', '0000070858', '0000078003', '0000002488', '0001045810', '0000707549', '0000884887', '0001065280', '0000912595', '0001

In [15]:
len(folder_names)

421